In [3]:
#LOAD IN DATA FROM EXCEL
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Read data
data_path = "nurse_schedule_project2_data_large_VA.xlsx"
locations = pd.read_excel(data_path, sheet_name="locations", index_col = 0, header = None)
nurses = pd.read_excel(data_path, sheet_name="nurses", index_col = 0)
task_time = pd.read_excel(data_path, sheet_name="task_execution_time", index_col = 0).T

patients = pd.read_excel(data_path, sheet_name="patients", index_col = 0)
# Break comma separated strings into lists
for col in ['M', 'T', 'W','Th','F','S','Su']:
    patients[col] = patients[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
for col in ['skillset']:
    nurses[col] = nurses[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Transpose the dataframe to access patient information easily
patients = patients.T
nurses = nurses.T


# Sets and Parameters
nurses = nurses.columns.tolist()            # set of nurses
patients = patients.columns.tolist()          # set of patients
locations = list(locations.index)         # set of locations
days = ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']  # set of days
T_tasks = task_time.columns.tolist() # set of tasks

# Parameters


# define a function to list all patients in a given location
def find_keys_with_inner_value(df, target_value):
    return list(df.T[df.T['location'] == target_value].index)


In [7]:
# City with needs on each day,
# array of 7 lists, each list contains patients in that city on that day
needs = []
for i in range(len(locations)):
    needs.append([])
    for j in range(len(days)):
        needs[i].append(find_keys_with_inner_value(patients, locations[i]))

print(needs)


# Total hours of needs on each day / city 



AttributeError: 'list' object has no attribute 'T'